In [1]:
using LinearAlgebra, Statistics, BenchmarkTools, SparseArrays, Random
Random.seed!(42); 

In [32]:
A = sprand(10, 10, 0.45)
@time invA = inv(Array(A))

B = sprand(10, 10, 0.45)
invB = sparse(inv(Array(B)))

  0.000036 seconds (6 allocations: 7.094 KiB)


10×10 SparseMatrixCSC{Float64,Int64} with 100 stored entries:
  [1 ,  1]  =  -0.191886
  [2 ,  1]  =  -0.293568
  [3 ,  1]  =  0.418896
  [4 ,  1]  =  1.5174
  [5 ,  1]  =  -0.0357174
  [6 ,  1]  =  -0.0930237
  [7 ,  1]  =  0.0790766
  [8 ,  1]  =  0.175746
  [9 ,  1]  =  0.00209055
  [10,  1]  =  -0.87469
  [1 ,  2]  =  -0.211962
  [2 ,  2]  =  0.900332
  ⋮
  [8 ,  9]  =  6.78016
  [9 ,  9]  =  2.61579
  [10,  9]  =  -3.71619
  [1 , 10]  =  -0.790739
  [2 , 10]  =  0.0162444
  [3 , 10]  =  -0.0443224
  [4 , 10]  =  0.182893
  [5 , 10]  =  0.0212251
  [6 , 10]  =  0.547007
  [7 , 10]  =  1.67983
  [8 , 10]  =  -1.6328
  [9 , 10]  =  -0.012293
  [10, 10]  =  -1.64688

In [33]:
using BenchmarkTools
function benchmark_solve(A, b)
    println("A\\b for typeof(A) = $(string(typeof(A)))")
    @btime $A \ $b
end


N = 1000
b = rand(N)
A = Tridiagonal([fill(0.1, N-2); 0.2], fill(0.8, N), [0.2; fill(0.1, N-2);])
A_sparse = sparse(A)  # sparse but losing tridiagonal structure
A_dense = Array(A)    # dropping the sparsity structure, dense 1000x1000

# benchmark solution to system A x = b
benchmark_solve(A, b)
benchmark_solve(A_sparse, b)
benchmark_solve(A_dense, b);

A\b for typeof(A) = Tridiagonal{Float64,Array{Float64,1}}
  23.514 μs (9 allocations: 47.75 KiB)
A\b for typeof(A) = SparseMatrixCSC{Float64,Int64}
  784.823 μs (69 allocations: 1.06 MiB)
A\b for typeof(A) = Array{Float64,2}
  16.977 ms (5 allocations: 7.65 MiB)


In fact, the difference becomes more extreme as the matrices grow. Solving a tridiagonal system is $O(N)$ while that of a dense matrix without any structure is $O(N^3)$. 

In [34]:
N = 5
U = UpperTriangular(rand(N,N))
L = U'
@time L * U


A = Tridiagonal([fill(0.1, N-2); 0.2], fill(0.8, N), [0.2; fill(0.1, N-2);])
D = Diagonal(rand(N))
@time D * A


  0.105660 seconds (147.26 k allocations: 7.369 MiB)
  0.609682 seconds (2.08 M allocations: 99.588 MiB, 4.55% gc time)


5×5 Tridiagonal{Float64,Array{Float64,1}}:
 0.586056   0.146514     ⋅          ⋅           ⋅ 
 0.0620506  0.496405    0.0620506   ⋅           ⋅ 
  ⋅         0.00307175  0.024574   0.00307175   ⋅ 
  ⋅          ⋅          0.0920293  0.736234    0.0920293
  ⋅          ⋅           ⋅         0.111219    0.444877

In [36]:
N = 100
A = rand(N,N)
M = 30
B = rand(N,M)
function solve_inverting(A, B)
    A_inv = inv(A)
    X = similar(B)
    for i in 1:size(B,2)
        X[:,i] = A_inv * B[:,i]
    end
    return X
end

function solve_factoring(A, B)
    X = similar(B)
    A = factorize(A)
    for i in 1:size(B,2)
        X[:,i] = A \ B[:,i]
    end
    return X
end



@btime solve_inverting($A, $B)
@btime solve_factoring($A, $B)

# even better, use built-in feature for multiple RHS
@btime $A \ $B;

  405.866 μs (68 allocations: 205.28 KiB)
  228.978 μs (96 allocations: 155.59 KiB)
  148.140 μs (6 allocations: 102.63 KiB)


In [37]:
N = 10
M = 3
x_true = rand(3)

A = rand(N,M) .+ randn(N)
b = rand(N)
x = A \ b

3-element Array{Float64,1}:
 -2.090019248085796
  0.9049883755395551
  1.2806259962712583

### Exercise 2a,b

#### Solution

In [100]:
N = 100
A = Tridiagonal([fill(0.1, N-2); 0.2], fill(0.8, N), [0.2; fill(0.1, N-2)])
A_adjoint = A';


# iterative calculation of  ψ_T

function iteration_1(T)
    ψ_old = [1; zeros(N-1)]
    for t in 1:T
        ψ_old = A' * ψ_old 
    end
    return ψ_old
end



# direct calculation of ψ_T

function iteration_2(T)
    ψ_old = [1; zeros(N-1)]
    return A_adjoint^T * ψ_old
end

# Eigen decomposition 

A = Array(A)
A_eig = eigen(A')
Λ = Diagonal(A_eig.values)
Q = A_eig.vectors

function iteration_3(T)
    ψ_old = [1; zeros(N-1)]
    return  Q * Λ^T * inv(Q)* ψ_old
end



T = 2N

@time iteration_1(T)
@time iteration_2(T)
@time iteration_3(T)


@show iteration_1(T) ≈ iteration_2(T)
@show iteration_1(T) ≈ iteration_3(T)
@show iteration_2(T) ≈ iteration_3(T);

  0.054605 seconds (19.56 k allocations: 1.187 MiB)
  0.019347 seconds (5.04 k allocations: 1.440 MiB)
  0.012553 seconds (3.99 k allocations: 482.340 KiB)
iteration_1(T) ≈ iteration_2(T) = true
iteration_1(T) ≈ iteration_3(T) = true
iteration_2(T) ≈ iteration_3(T) = true
